# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TIRUN package.
  * Set things up to carry out geometry optimization using Amber.

In [1]:
startSwank 4006

T

Checking SLIME_HOME
About to start swank server
Started swank server


In [2]:
quickload :tirun-jupyter

keyword str -> "TIRUN-JUPYTER"
To load "tirun-jupyter":
  Load 1 ASDF system:
    tirun-jupyter


(:TIRUN-JUPYTER)


; Loading "tirun-jupyter"



In [3]:
(leap:setup-amber-paths)

(#P"/Users/su-chris/Development/amber18/dat/antechamber/"
 #P"/Users/su-chris/Development/amber18/dat/leap/cmd/oldff/"
 #P"/Users/su-chris/Development/amber18/dat/leap/cmd/"
 #P"/Users/su-chris/Development/amber18/dat/leap/parm/"
 #P"/Users/su-chris/Development/amber18/dat/leap/lib/"
 #P"/Users/su-chris/Development/amber18/dat/leap/prep/")

# Start a new TIRUN calculation

In [4]:
(tirun-jupyter:new-tirun)

Jobs will be saved to /Users/su-chris/jobs/
setenv CANDO_JOBS_DIRECTORY if you want it to go elsewhere


In [ ]:
(tirun-jupyter::load-tirun)

Loading application state from default.dat...


SIMPLE-PROGRAM-ERROR: Could not find field :BL

Could not find field :BL
   [Condition of type CORE:SIMPLE-PROGRAM-ERROR]

#<DISSECT:ENVIRONMENT>
   [Environment of thread #<PROCESS "SHELL Thread" @0x14207ea31 (Running)>]


Backtrace:
 7: (#<FUNCTION CORE:SIGNAL-SIMPLE-ERROR> SIMPLE-PROGRAM-ERROR NIL Could not find field :BL NIL)
 14: (#<FUNCTION CORE:LOAD-CXX-OBJECT> RESIDUE (CONTENTS . #(#<ATOM :N1/:N 0xe1ea1f30bf5f0073> #<ATOM :C2/:C 0x73e016d1ba2d7405> #<ATOM :C3/:C 0xb58ed3f38abdc32f> #<ATOM :C4/:C 0xabce196b58c82215> #<ATOM :C5/:C 0x82aa0e321b4e8316> #<ATOM :C6/:C 0x312bab0aaa1540f5> #<ATOM :O7/:O 0xde8c0fbe9416d6d7> #<ATOM :C8/:C 0xcf337d6c857865f2> #<ATOM :N9/:N 0x624ce1d9fc8c4115> #<ATOM :C10/:C 0x28bf0b7e0ce1f86> #<ATOM :C11/:C 0xf9e318d4844a412d> #<ATOM :C12/:C 0x913d153a568354a6> #<ATOM :C13/:C 0xfd05809cf7e66f92> #<ATOM :C14/:C 0x3c3f666890558b6b> #<ATOM :C15/:C 0x3016b232a842b6d> #<ATOM :C16/:C 0x6844fc00652a96a5> #<ATOM :C17/:C 0xc0ad0ca757418c97> #<ATOM :O18/:O 0xd8fb803996936dbb> #<ATOM :N19/:N 0x4ac21727eb62bb32> #

(define-symbol-macro all-ligands (tirun-jupyter::all-ligands tirun-jupyter::*app*))
(define-symbol-macro selected-ligands (tirun-jupyter::selected-ligands tirun-jupyter::*app*))
(define-symbol-macro tiruns tirun-jupyter::*tirun*)

# Load the Thrombin PDB file

In [5]:
(tirun-jupyter:load-receptor)

A Jupyter Widget

A Jupyter Widget

In [6]:
(tirun-jupyter::tirun-calculation tirun-jupyter::*app*)

#<tirun-calculation>

In [7]:
agg = (first (tirun::receptors (tirun-jupyter::tirun-calculation tirun-jupyter::*app*)))

#<AGGREGATE unknown 0x9593b4491680096b>

In [8]:
mol = (chem:content-at agg 0)

#<MOLECULE :mol_0 0xb92efd1ab57310cc>

In [9]:
show mol

A Jupyter Widget

In [10]:
(cando:save-cando mol "/tmp/mol.cando")

#<MOLECULE :mol_0 0xb92efd1ab57310cc>

In [11]:
mol1 = (cando:load-cando "/tmp/mol.cando")

#<MOLECULE :mol_0 0x6f4ec01092c997bc>

In [17]:
(chem:number-of-atoms mol1)

5474

In [18]:
(chem:number-of-atoms mol)

5474

In [14]:
v = (show mol1)

A Jupyter Widget

In [16]:
(nglv:add-representation v "ball+stick")

T

In [ ]:
(tirun-jupyter::load-template-ligand)

In [ ]:
(tirun::template-ligand (tirun-jupyter::tirun-calculation tirun-jupyter::*app*))

# Sketch the ligands using Kekule composer

In [ ]:
(tirun-jupyter:composer)

In [ ]:
show (third all-ligands)

# Import the sketch from a Chemdraw file - skip if you used kekule above

In [ ]:
(jupyter:png-file "ligand3.png" )

In [ ]:
sketch = (tirun:load-chem-draw-tirun "ligand3.cdxml")

In [ ]:
(tirun::assemble-ligands tiruns sketch)

In [ ]:
(tirun::pose-ligands-using-similarity tiruns ligmol)

In [ ]:
all-ligands = (mapcar #'tirun::molecule (tirun::ligands tiruns))

In [ ]:
selected-ligands = all-ligands

# The two approaches rejoin here.  The ligands are in tiruns and in all-ligands and selected-ligands

In [ ]:
(tirun-jupyter:select-ligands)

In [ ]:
selected-ligands

In [ ]:
(length selected-ligands)

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(tirun-jupyter:lomap)

In [ ]:
(tirun-jupyter:configure-jobs)

In [ ]:
(tirun-jupyter:write-jobs)